<img src="../assets/logos/Logo_shoot.png" width=500></img>
# Second Model: Probability
Instead of shooting randomly, like the first model, lets make a model who guesses based on past games! This model will first make 10 random shots at the begining of the game. The model will then look at past games who had the same hits/misses as the 10 shots and choose shots based on the location of ships in those games. If no past games match then additional random shots will be taken. 

## Data
We will be using past game date to train out model. First we need to download the data by running: (note the data is constantly being updated as games are played)
```bash
./bin/download_data.sh
```
next copy the sample data file into the mock directory of the code for this model
```bash
cp ./data/data.min.json ./containers/Shoot/Bayes/mock/data/data.json
```
You will now be able to test your model training localy 

Next we need to upload the data to s3 so we can train the production model on it. run:
```bash
./bin/upload_data.sh
```

## Code
1. code is in /containers/shoot
1. train.py parses the game data and saves all the layouts
1. host.py takes the saved layouts and finds all layouts that match the current game board and chooses a random ship location from the matched layouts
1. if no match then randomly shoot

## Configure
Before you can deploy your second model you need to configure the deployment. As described in the [Endpoint Notebook](../Endpoint_Reference.ipynb), you will edit to the deployment configuration in /deploy-config.json. you need to edit the following field to the following values

|parameter field|Value|
|---|---|
|trainsourcefile|`shoot-Bayes.tar.gz`|
|hostsourcefile|`shoot-Bayes.tar.gz`|
|channels|{train:{path:"all/data.json"}}|

## Deploy
1. follow instructions from endpoint reference to commit your code and changes and then push your changes to the remote branch
2. tell opperations to deploy your changes

## Amazon Athena
You will use Amazon Athena to monitor the performance of your model. Athena allows you to run SQL queries against data in s3, like the game archive data. You can see a example query by running the following command:

In [1]:
!../../bin/glue_info.sh

{
  "GameDatabase": "datacatalog-bnxdo93q77sg",
  "GameTable": "gameday-main-9-gamearchivebucket-ce4fiv1vk6n8"
}
----example sql Query----

SELECT * FROM datacatalog-bnxdo93q77sg.gameday-main-9-gamearchivebucket-ce4fiv1vk6n8 limit 10;



1. Go to the [Athena Console](https://console.aws.amazon.com/athena/home?region=us-east-1#query)
1. you are going to use the following query but you need to first get some values. You need the name of the Amazon Glue catalog that holds the game data and the s3 bucket where that data is stored. These values are given by the above command. Replace <Game-data-catalog> and <Game-data-bucket> with those values. so for example the line `FROM "<Game-data-catalog>"."<Game-data-bucket>"` will look something like `FROM "game-catalog-adafafdad"."game-archive-adfasdfasdf"`
1. Next replace the three occurances of <Team-Name> with your team name. For example, the line `WHEN winner = '<Team-Name>' THEN 1.0` becomes `WHEN winner = 'my-team' THEN 1.0`

```sql
SELECT type,
         sum(won)/(sum(won)+sum(lost)) as won,
         sum(lost)/(sum(won)+sum(lost)) as lost
FROM 
    (SELECT json_extract(session,'$.type') AS type,
        CASE
            WHEN winner = '<Team-Name>' THEN 1.0
            ELSE 0.0
        END AS won,
        CASE
            WHEN winner != '<Team-Name>' THEN 1.0
            ELSE 0.0
        END AS lost
    FROM 
        (SELECT teama.session AS session,
         teama.teamname AS team,
         winner
        FROM "<Game-data-catalog>"."<Game-data-bucket>"
        UNION
        SELECT teamb.session AS session,
         teamb.teamname AS team,
         winner
        FROM "<Game-data-catalog>"."<Game-data-bucket>" )
        WHERE team = '<Team-Name>' )
    GROUP BY  size
```

## next Model
Next, read through [Third Notebook](./Third_Deploy.ipynb) for how to configure your first model